<a href="https://colab.research.google.com/github/nyanta012/demo/blob/main/faster_whisper_v4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
# ランタイム⇒ランタイムのタイプの変更でGPUをオンにする必要があります

In [10]:
!pip install "faster-whisper @ https://github.com/nyanta012/faster-whisper/archive/refs/heads/master.tar.gz"
!pip install transformers==4.27.2

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/nyanta012/faster-whisper/archive/refs/heads/master.tar.gz (13 kB)
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
# srt file を作成するための関数
def create_file(file_name="transcribe", results=None, fast_whisper=True, file_type="srt"):
    if not file_type in ["srt", "txt"]:
        raise NotImplementedError("ファイル拡張子は[srt, txt]を選択してください。")
    with open(f"{file_name}.{file_type}", mode="w") as f:
        for index, _dict in enumerate(results):
            if fast_whisper:
                start_time = _dict[0] # start
                end_time = _dict[1] # end
                text = _dict[2] # text
            else:
                start_time = _dict["start"]
                end_time = _dict["end"]
                text = _dict["text"]
            s_h, e_h = int(start_time//(60 * 60)), int(end_time//(60 * 60))
            s_m, e_m = int((start_time % (60 * 60))//60), int((end_time % (60 * 60))//60)
            s_s, e_s = int(start_time % 60), int(end_time % 60)
            if file_type=="srt":
                f.write(f'{index+1}\n{s_h:02}:{s_m:02}:{s_s:02},000 --> {e_h:02}:{e_m:02}:{e_s:02},000\n{text}\n\n')
            elif file_type=="txt":
                f.write(f'[{s_h:02}:{s_m:02}:{s_s:02} ===> {e_h:02}:{e_m:02}:{e_s:02}] {text}\n')

In [24]:
# アップロード後、下記を実行することで、ファイル名を取得
import os
os.listdir(".")

['.config',
 'whisper-large-v2-ct2',
 'transcribe_faster_whisper.txt',
 'サンプル動画.mkv',
 'transcribe_faster_whisper.srt',
 'sample_data']

In [25]:
# アップロードしたファイル名を設定(拡張子まで含める)
file_name = "サンプル動画.mkv"

下記はFaster Whisperを使用する場合

In [ ]:
# whisper large-v2をhugging faceから取得して、ctranslate2でC++に変換&パラメータなどの量子化する
from faster_whisper import WhisperModel
!ct2-transformers-converter --model openai/whisper-large-v2 --output_dir whisper-large-v2-ct2 \
    --copy_files tokenizer.json --quantization float16

model_path = "whisper-large-v2-ct2/"

In [26]:
# Run on GPU with FP16
model = WhisperModel(model_path, device="cuda", compute_type="float16")

In [30]:
%%time
results, _ = model.transcribe(file_name,
                              language="ja")

CPU times: user 131 ms, sys: 10.2 ms, total: 141 ms
Wall time: 131 ms


In [31]:
create_file(file_name="transcribe_faster_whisper", results=results, fast_whisper=True, file_type="srt")

下記は本家whisperのlarge-v2を使用する場合

In [32]:
# 本家whisperのlarge-v2を使用する場合

In [33]:
!pip install git+https://github.com/nyanta012/whisper_v2.git 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/nyanta012/whisper_v2.git to /tmp/pip-req-build-0pnqwkt0
  Running command git clone --filter=blob:none --quiet https://github.com/nyanta012/whisper_v2.git /tmp/pip-req-build-0pnqwkt0
  Resolved https://github.com/nyanta012/whisper_v2.git to commit 6dea21fd7f7253bfe450f1e2512a0fe47ee2d258
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.3/63.3 MB 13.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 KB 19.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20230314-py3-none-any.whl size=796926 sha256=ef0a4c96def6a356aa7563096051bba0bcdc30887ce2b2858c4edd30e8b2a

In [34]:
import whisper

model = whisper.load_model("large-v2")

100%|█████████████████████████████████████| 2.87G/2.87G [01:05<00:00, 47.2MiB/s]


In [37]:
%%time
segments = model.transcribe(file_name, language="en")

CPU times: user 3.47 s, sys: 137 ms, total: 3.6 s
Wall time: 4.47 s


In [40]:
create_file(file_name="transcribe_openai_whisper_en", results=segments["segments"], fast_whisper=False, file_type="srt")